# Tutorial on Variable Ordering in FXSDD(BR)

The aim of this brief tutorial is to provide some insights into how the FXSDD(BR) solver represents and evaluates WMI problems, and to show the importance of variable ordering.
This work originates from the UAI2020 paper by Derkinderen et al. [Ordering Variables for Weighted Model Integration](https://lirias.kuleuven.be/3060810?limo=0).


If you are simply interested in changing the variable ordering heuristic: go to the last 'section'.

Before we get started, we need some imports and helper functions...

In [9]:
import graphviz
from pywmi import Domain, Density, PyXaddAlgebra
from pywmi.engines.pyxadd.engine import ToXaddWalker
from pywmi.engines.pyxadd.operation import Multiplication
from pywmi.engines.xsdd.draw import sdd_to_dot
from pywmi.engines.xsdd.smt_to_sdd import compile_to_sdd
from pywmi.engines.xsdd.literals import extract_and_replace_literals
from pywmi.engines.xsdd.vtrees.vtree import bami, balanced, leftlinear, rightlinear
from pywmi.engines.xsdd import FactorizedXsddEngine as FXSDD
import pysmt.shortcuts as smt


def get_support_sdd(logic_support, literals, vtree_strategy=bami):
    vtree = vtree_strategy(literals)
    support_sdd = compile_to_sdd(logic_support, literals, vtree)
    return support_sdd


def view_support_sdd(support, vtree_strategy=bami):
    repl_env, logic_support, literals = extract_and_replace_literals(support)
    support_sdd = get_support_sdd(logic_support, literals, vtree_strategy)
    support_dot = sdd_to_dot(support_sdd, literals, draw_false=False, draw_single_or=False)
    #graphviz.Source(support_dot).render(view=True)
    return graphviz.Source(support_dot)

def get_literal_obj(support):
    # This is how the FXSDD(BR) solver would create it.
    # Warning: FXDD(BR) solver considers a problem several pieces.
    # Such problems would have multiple calls, creating multiple literal objects.
    repl_env, logic_support, literals = extract_and_replace_literals(support)
    return literals


def integrate_one_var(problem, first_var):
    algebra = PyXaddAlgebra(reduce_strategy=PyXaddAlgebra.FULL_REDUCE)
    # Construct Theory in XADD format
    theory_xadd = ToXaddWalker(True, algebra.pool).walk_smt(problem.support)
    weight_xadd = ToXaddWalker(False, algebra.pool).walk_smt(problem.weight)
    combined = algebra.pool.apply(Multiplication, theory_xadd, weight_xadd)
    # Integrate out variable
    result = algebra.integrate(problem.domain, combined, [first_var])
    # Show results
    xadd = algebra.pool.diagram(result)
    xadd.show(True)

## FXSDD(BR)

The FXSDD(BR) solver uses (multiple) Sentential Decision Diagrams (SDD) to represent the problem's support. Consider the following problem:

In [10]:
n = 4
domain = Domain.make([], [f"x{i}" for i in range(n)], real_bounds=(0, 1))
x = domain.get_symbols()
support = (x[0] <= x[1]) & (x[0] <= x[2]) & (x[0] <= x[3])
weight = smt.Real(1)
problem = Density(domain, support & domain.get_bounds(), weight)
print(f"Domain: {domain}")
print(f"Support: {support}")
print(f"Weight: {weight}")

Domain: (x0[0, 1], x1[0, 1], x2[0, 1], x3[0, 1])
Support: (((x0 <= x1) & (x0 <= x2)) & (x0 <= x3))
Weight: 1.0


The support of this problem could be represented using SDDs as seen below, where AND- and OR-nodes represent product and sum-operations respectively. For those familiar with visualizing SDDs, we ommitted all nodes evaluating to false as well as all OR-nodes with a single-child.

In [11]:
view_support_sdd(problem.support, vtree_strategy=bami)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

FXSDD(BR) computes the volume by evaluating this equation (SDD structure) bottom-up, intermediately integrating out continuous variables.
Concerning the integration, FXSDD(BR) pushes them as close to the leafs as possible to obtain smaller intermediate results (that could be reused by a caching mechanism).
This is possible because integration can be pushed through AND-nodes if the continuous variable is only present in one branch, and can always be pushed through OR-nodes.
In our previous example, x1, x2 and x3 can all be pushed further below the top AND-node.


## Ordering importance

The extend in which the integration of continuous variables can be pushed lower into the structure impacts
the size of the intermediate results which in their turn influence the size of future operations and
hence the performance of the FXSDD(BR) solver.
This means that ideally, the SDD is structured such that AND-node branches share as few continuous variables as possible.

Another aspect that influences the size of intermediate results and hence the performance is the order of integration.
Consider our previous problem statement. First integrating out x0 would result in an intermediate result much larger than when we first integrate out other variables such as x1.

In [12]:
integrate_one_var(problem, first_var='x0')

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Source.gv'], make sure the Graphviz executables are on your systems' PATH

In [ ]:
integrate_one_var(problem, first_var='x1')

The reason that first integrating out x1 results in a smaller intermediate result compared to first integrating out x0 has to do with the interactions of the variables.
When you look at the initial problem statement, you might notice that x0 occurs in inequalities with all other continuous variables
while the other continuous variables only co-occur with x0. By integrating out x0, we create connections between all the other variables.

So ideally, the structure of an SDD allows for the integration to be pushed deep into the structure, and allows for an integration order that yields small intermediate results.

## FXSDD(BR) + custom variable ordering heuristics

The following code uses the FXSDD(BR) solver to compute the volume of our problem.
Notice the vtree_strategy parameter of the FXSDD constructor. Heuristics originating from the SDD package are balanced, leftlinear and rightlinear.
The bami heuristic used in the code below is an heuristic introduced by Derkinderen et al. in UAI2020, '[Ordering Variables for Weighted Model Integration](https://lirias.kuleuven.be/3060810?limo=0)'.
In contrast to the three previous heuristic, Bami analyses the problem, more specifically the continuous variable interactions, to obtain a better vtree and hence SDD.

In [ ]:
algebra = PyXaddAlgebra(reduce_strategy=PyXaddAlgebra.FULL_REDUCE)
mfxsdd = FXSDD(
    problem.domain,
    problem.support,
    problem.weight,
    vtree_strategy=bami,
    #vtree_strategy=balanced,
    #vtree_strategy=leftlinear,
    #vtree_strategy=rightlinear,
    algebra=algebra,
    ordered=False,
)
volume = mfxsdd.compute_volume(add_bounds=False)
print(volume)

To try out your own heuristic, simply create a new method and pass it instead of bami.
This heuristic method must take as input a LiteralInfo object and return an appropriate Vtree.

In [ ]:
literal_info = get_literal_obj(problem.support)
vtree = bami(literal_info)
for lit_item in literal_info.literals.items():
    print(lit_item)
graphviz.Source(vtree.to_dot()) #.render(view=True)